In [1]:
import sys
!{sys.executable} -m pip install openpyxl



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip


In [2]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/raw")  # or Path(".") if all files are beside the notebook

yearly_team_off = pd.read_csv(DATA_DIR / "yearly_team_stats_offense.csv")
yearly_team_def = pd.read_csv(DATA_DIR / "yearly_team_stats_defense.csv")

yearly_player_off = pd.read_csv(DATA_DIR / "yearly_player_stats_offense.csv")
yearly_player_def = pd.read_csv(DATA_DIR / "yearly_player_stats_defense.csv")

weekly_team_off = pd.read_csv(DATA_DIR / "weekly_team_stats_offense.csv")
weekly_team_def = pd.read_csv(DATA_DIR / "weekly_team_stats_defense.csv")

weekly_player_off = pd.read_csv(DATA_DIR / "weekly_player_stats_offense.csv")
weekly_player_def = pd.read_csv(DATA_DIR / "weekly_player_stats_defense.csv")

cap_tables = pd.read_csv(DATA_DIR / "2014-thru-2020-cap-tables.csv")
pos_group = pd.read_excel(DATA_DIR / "NFL Salary By Position Group.xlsx")

print("yearly_team_off:", yearly_team_off.shape)
print("yearly_team_def:", yearly_team_def.shape)
print("pos_group      :", pos_group.shape)


yearly_team_off: (582, 99)
yearly_team_def: (582, 47)
pos_group      : (320, 33)


In [3]:
# Only use regular season data
off_reg = yearly_team_off[yearly_team_off["season_type"] == "REG"].copy()
def_reg = yearly_team_def[yearly_team_def["season_type"] == "REG"].copy()

print(off_reg.shape, def_reg.shape)


(416, 99) (416, 47)


In [4]:
#merge offense and defense
team_stats = off_reg.merge(
    def_reg,
    on=["team", "season", "season_type"],
    suffixes=("_off", "_def")
)

print("Merged team_stats shape:", team_stats.shape)
team_stats.head()


Merged team_stats shape: (416, 143)


,team,season,season_type,shotgun,no_huddle,qb_dropback,qb_scramble,total_off_yards,pass_attempts,complete_pass,...,average_tackle_with_assist,average_sack,average_qb_hit,average_def_touchdown,average_defensive_two_point_attempt,average_defensive_two_point_conv,average_defensive_extra_point_attempt,average_defensive_extra_point_conv,rush_pct_def,pass_pct_def
0,ARI,2012,REG,605.0,56.0,676.0,12.0,4587,587.0,337.0,...,2.625000,2.375000,5.250000,0.250000,0.0,0.0,0.0,0.0,0.47,0.51
1,ATL,2012,REG,418.0,101.0,664.0,20.0,6116,601.0,422.0,...,1.388889,1.777778,4.111111,0.111111,0.0,0.0,0.0,0.0,0.41,0.59
2,BAL,2012,REG,452.0,257.0,608.0,9.0,5897,549.0,334.0,...,8.050000,2.300000,5.400000,0.150000,0.0,0.0,0.0,0.0,0.45,0.55
3,BUF,2012,REG,666.0,34.0,571.0,31.0,5647,494.0,309.0,...,7.562500,2.250000,3.937500,0.062500,0.0,0.0,0.0,0.0,0.44,0.55
4,CAR,2012,REG,711.0,35.0,559.0,36.0,5989,478.0,284.0,...,1.437500,2.437500,3.937500,0.187500,0.0,0.0,0.0,0.0,0.40,0.59


In [5]:
#clean data
pos_group["season"] = pos_group["Team_Season"].str[-4:].astype(int)

# Add games per season (16 before 2021, 17 from 2021 onward)
def games_in_season(s):
    return 17 if s >= 2021 else 16

pos_group["games"] = pos_group["season"].apply(games_in_season)
pos_group["win_pct"] = pos_group["W"] / pos_group["games"]

pos_group[["Team_Season", "Team", "season", "W", "win_pct"]].head()


,Team_Season,Team,season,W,win_pct
0,Cardinals 2022,Cardinals,2022,4,0.235294
1,Falcons 2022,Falcons,2022,7,0.411765
2,Ravens 2022,Ravens,2022,10,0.588235
3,Bills 2022,Bills,2022,13,0.764706
4,Panthers 2022,Panthers,2022,7,0.411765


In [6]:
#abbreviation mapping
team_abbr_to_name = {
    "ARI": "Cardinals",
    "ATL": "Falcons",
    "BAL": "Ravens",
    "BUF": "Bills",
    "CAR": "Panthers",
    "CHI": "Bears",
    "CIN": "Bengals",
    "CLE": "Browns",
    "DAL": "Cowboys",
    "DEN": "Broncos",
    "DET": "Lions",
    "GB":  "Packers",
    "HOU": "Texans",
    "IND": "Colts",
    "JAX": "Jaguars",
    "KC":  "Chiefs",
    "LAC": "Chargers",
    "LAR": "Rams",
    "MIA": "Dolphins",
    "MIN": "Vikings",
    "NE":  "Patriots",
    "NO":  "Saints",
    "NYG": "Giants",
    "NYJ": "Jets",
    "LV":  "Raiders",   # Oakland/Las Vegas — simplified
    "OAK": "Raiders",
    "PHI": "Eagles",
    "PIT": "Steelers",
    "SEA": "Seahawks",
    "SF":  "49ers",
    "TB":  "Buccaneers",
    "TEN": "Titans",
    "WAS": "Commanders",  # old Washington team
    "WSH": "Commanders"
}

team_stats["team_name"] = team_stats["team"].map(team_abbr_to_name)

# Quick sanity check
team_stats[["team", "team_name"]].drop_duplicates().head(10)


,team,team_name
0,ARI,Cardinals
1,ATL,Falcons
2,BAL,Ravens
3,BUF,Bills
4,CAR,Panthers
5,CHI,Bears
6,CIN,Bengals
7,CLE,Browns
8,DAL,Cowboys
9,DEN,Broncos


In [7]:
#merge data
team_merged = team_stats.merge(
    pos_group,
    left_on=["team_name", "season"],
    right_on=["Team", "season"],
    how="inner"
)

print("team_merged shape:", team_merged.shape)
team_merged[["team", "team_name", "season", "W", "win_pct"]].head()


team_merged shape: (310, 179)


,team,team_name,season,W,win_pct
0,ARI,Cardinals,2013,10,0.625
1,ATL,Falcons,2013,4,0.250
2,BAL,Ravens,2013,8,0.500
3,BUF,Bills,2013,6,0.375
4,CAR,Panthers,2013,12,0.750


In [8]:

# Target variable
target_col = "win_pct"
y = team_merged[target_col]


drop_from_X = [
    "win_pct",   # target
    "W",         # leaks target
    "Team_Season"
]

# X will contain ALL useful numeric features
X = team_merged.drop(columns=drop_from_X)

# Keep dataset intact for longitudinal analysis
print("\nX shape (features):", X.shape)
print("y shape (target):", y.shape)




X shape (features): (310, 176)
y shape (target): (310,)


In [9]:
#Save data 
OUTPUT_DIR = Path("../data/processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

team_merged.to_csv(OUTPUT_DIR / "team_season_merged.csv", index=False)

print("\nSaved: data/processed/team_season_merged.csv")


Saved: data/processed/team_season_merged.csv
